## Import modules and create a spark session

In [3]:
#import modules
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

#create Spark session
appName = "Regression in Spark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option","some-value") \
    .getOrCreate()

## Read data file into Spark dataFrame

In [6]:
#define our schema
flightSchema = StructType([
    StructField("DayOfMonth", IntegerType(), False),
    StructField("DayOfWeek", IntegerType(), False),
    StructField("Carrier", StringType(), False),
    StructField("OriginAirportID", IntegerType(), False),
    StructField("DestAirportID", IntegerType(), False),
    StructField("DepDelay", IntegerType(), False),
    StructField("ArrDelay", IntegerType(), False),
])

#read csv data with our defined schema
flightDataFrame = spark.read.csv('dataset/flights.csv', schema=flightSchema, header=True)
flightDataFrame.show(3)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayOfMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
+----------+---------+-------+---------------+-------------+--------+--------+
only showing top 3 rows



## Select only important columns for regression features

In [7]:
#select columns
data = flightDataFrame.select("DayOfMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay", "ArrDelay")
data.show(3)

+----------+---------+---------------+-------------+--------+--------+
|DayOfMonth|DayOfWeek|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+---------------+-------------+--------+--------+
|        19|        5|          11433|        13303|      -3|       1|
|        19|        5|          14869|        12478|       0|      -8|
|        19|        5|          14057|        14869|      -4|     -15|
+----------+---------+---------------+-------------+--------+--------+
only showing top 3 rows



## Divide data into training and testing data

In [8]:
#divide data into 70% for training and 30% for testing
dividedData = data.randomSplit([0.7,0.3])
trainingData = dividedData[0] #index-0 from dividedData
testingData = dividedData[1] #index-1 from dividedData
train_rows = trainingData.count()
test_rows = testingData.count()

print("Training data rows:", train_rows, "; Testing data rows:", test_rows)

Training data rows: 1891212 ; Testing data rows: 811006


## Prepare training data

In [10]:
#define an assambler
assambler = VectorAssembler(inputCols = [
    "DayOfMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay"
], outputCol="features")

#change our features into one column using our defined assambler
trainingDataFinal = assambler.transform(trainingData).select(col("features"),(col("ArrDelay").cast("Int").alias("label")))
trainingDataFinal.show(truncate=False, n=4)

+------------------------------+-----+
|features                      |label|
+------------------------------+-----+
|[1.0,1.0,10140.0,10821.0,8.0] |-9   |
|[1.0,1.0,10140.0,11259.0,-1.0]|-11  |
|[1.0,1.0,10140.0,11259.0,0.0] |-12  |
|[1.0,1.0,10140.0,11259.0,0.0] |-10  |
+------------------------------+-----+
only showing top 4 rows



## Train our regression model using training data

In [11]:
#call Spark Linear regression we import before
algoritma = LinearRegression(labelCol="label", featuresCol="features", maxIter=10, regParam=0.3)
#train the model
model = algoritma.fit(trainingDataFinal)
print("Regression model is trained!")

Regression model is trained!


## Prepare testing data

In [12]:
#change our feature data into one column using our defined assambler
#just like what we did before in training data
testingDataFinal = assambler.transform(testingData).select(col("features"),(col("ArrDelay").cast("Int").alias("trueLabel")))
testingDataFinal.show(truncate=False, n=4)

+------------------------------+---------+
|features                      |trueLabel|
+------------------------------+---------+
|[1.0,1.0,10140.0,10397.0,-4.0]|-11      |
|[1.0,1.0,10140.0,10397.0,-2.0]|-17      |
|[1.0,1.0,10140.0,11259.0,-3.0]|-11      |
|[1.0,1.0,10140.0,11259.0,-2.0]|-14      |
+------------------------------+---------+
only showing top 4 rows



## Predict the testing data using our trained model

In [15]:
#predict testing data using our model
prediction = model.transform(testingDataFinal)
#choose the columns
finalPrediction = prediction.select("features","trueLabel","prediction")
#show some prediction results
finalPrediction.show(10)

+--------------------+---------+-------------------+
|            features|trueLabel|         prediction|
+--------------------+---------+-------------------+
|[1.0,1.0,10140.0,...|      -11| -7.549327704638504|
|[1.0,1.0,10140.0,...|      -17| -5.553231715045605|
|[1.0,1.0,10140.0,...|      -11|   -6.7513555270562|
|[1.0,1.0,10140.0,...|      -14|-5.7533075322597504|
|[1.0,1.0,10140.0,...|       19| 17.201796348058593|
|[1.0,1.0,10140.0,...|       -6| -4.762919052507462|
|[1.0,1.0,10140.0,...|       -5|-3.7648710577110123|
|[1.0,1.0,10140.0,...|       -1|-1.7687750681181127|
|[1.0,1.0,10140.0,...|      -25| -9.754551665588648|
|[1.0,1.0,10140.0,...|       -9|   -4.7643116916064|
+--------------------+---------+-------------------+
only showing top 10 rows



## Calculate how accurate our model performance

In [17]:
#import evaluator modlue for regression
from pyspark.ml.evaluation import RegressionEvaluator

#define our evaluator
evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
#calculate RMSE of our trainde model
rmse = evaluator.evaluate(prediction)
print("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): 13.285331259455445
